In [1]:
import pandas
import numpy as np
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
import time
import pickle

import time
%matplotlib inline
from random import randint
from annoy import AnnoyIndex

In [2]:
dataset_name = "data/raw/data_1.csv"
data = pandas.read_csv(dataset_name).drop(["id1", "id2"], axis=1)
data.head()

,id3,user_id,date
0,714,464300,1
1,590,1029729,1
2,703,861305,1
3,803,733016,1
4,219,448027,1


In [3]:
data = data[data["user_id"].isin(data["user_id"].unique()[:53979])][["user_id", "id3"]].get_values()

In [4]:
users = np.unique(data[:,0])
users_vectors = dict([(user, np.zeros(931)) for user in users])

In [5]:
def ohe(id3):
    vector = np.zeros(931)
    vector[id3] = 1
    
    return vector

In [6]:
for row in data:
    users_vectors[row[0]] += ohe(row[1])

In [7]:
for user in users:
    vec = users_vectors[user]
    users_vectors[user] = vec/vec.sum()

In [8]:
ann = AnnoyIndex(931)

In [9]:
for user in users:
    ann.add_item(user, users_vectors[user])

In [10]:
ann.build(10)

True

In [11]:
items = [i for i in range(931)]

In [12]:
recomendations = []
for uid in users_vectors:
    vec = np.mean([users_vectors[i] for i in ann.get_nns_by_item(uid, 50)[1:]],axis=0)
    vec[users_vectors[uid] > 0] = 0
    row = [uid] + [i[1] for i in sorted(list(zip(vec.tolist(), items)), reverse=True)[:5]]
    
    recomendations.append(row)

In [13]:
submit = pandas.DataFrame(data=recomendations, columns=["user_id", "id3_1", "id3_2", "id3_3", "id3_4", "id3_5"])

In [14]:
submit.to_csv("submit.csv", index=False)

In [15]:
submit.head()

,user_id,id3_1,id3_2,id3_3,id3_4,id3_5
0,27,142,69,711,528,411
1,44,872,250,72,142,134
2,53,329,883,822,718,612
3,56,864,340,224,569,329
4,60,875,204,586,51,41
